In [1]:
import numpy as np 
from sklearn.model_selection import KFold
import pandas as pd
import sys
sys.path.append("../")
import gpflow
import tensorflow as tf
from gpsampling import GPActive
import xgboost
import multiprocessing

/home/nipun.batra/anaconda3/envs/gpflow/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/nipun.batra/anaconda3/envs/gpflow/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/nipun.batra/anaconda3/envs/gpflow/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/nipun.batra/anaconda3/env

In [3]:
def active(train_days, i, j, isact):
    df = pd.read_csv("../../data/beijingb_scaled.csv", index_col = 0)
    df = df.rename(columns={'ts': 'Time', 'station_id': 'Station'})

    stations = df['Station'].unique()
    stations.sort()

    splits = 6

    kfout = KFold(n_splits=splits, shuffle=True, random_state=0)
    kfin = KFold(n_splits=splits - 1, shuffle=True, random_state=0)

    station_split = {}

    for kout, (sts_ftrain_index, sts_test_index) in enumerate(kfout.split(stations)):
        station_split[kout] = {}
        for kin, (sts_train_index, sts_val_index) in enumerate(kfin.split(sts_ftrain_index)):
            sts_test = stations[sts_test_index]
            sts_val = stations[sts_ftrain_index[sts_val_index]]
            sts_train = stations[sts_ftrain_index[sts_train_index]]
            station_split[kout][kin] = {
                'test': sts_test,
                'train': sts_val,
                'pool': sts_train,
            }

    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    session = tf.Session(config=config)


    gp = GPActive(
                    df = df,
                    train_stations = list(station_split[i][j]['train']),
                    pool_stations = list(station_split[i][j]['pool']),
                    test_stations = list(station_split[i][j]['test']),
                    context_days = train_days - 1 , # train_days - 1
                    frequency = 30, # 
                    test_days = 365 - train_days,  # 
                    train_days = train_days, # 10, 20, 30
                    number_to_query = 1,
                    number_of_seeds= 5,
                    fname=[i, j]
                )
    print(isact)
    if isact == 1:
        gp.active_gp()
    else:
        gp.random_sampling()

In [ ]:
active(20, 2, 1, 1)

INIT DONE
True True
INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 471.150772
  Number of iterations: 352
  Number of functions evaluations: 400


INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 471.150772
  Number of iterations: 352
  Number of functions evaluations: 400


1
Current Day before update: 19

Current Day after update: 20
Stations added -  [1031.0]  on day  20
INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 477.918159
  Number of iterations: 125
  Number of functions evaluations: 141


INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 477.918159
  Number of iterations: 125
  Number of functions evaluations: 141


Current Day before update: 20

Current Day after update: 21
INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 479.133019
  Number of iterations: 326
  Number of functions evaluations: 385


...

INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 1277.806303
  Number of iterations: 71
  Number of functions evaluations: 78


Current Day before update: 223

Current Day after update: 224
INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 1278.343011
  Number of iterations: 62
  Number of functions evaluations: 109


INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 1278.343011
  Number of iterations: 62
  Number of functions evaluations: 109


Current Day before update: 224

Current Day after update: 225
INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 1279.671642
  Number of iterations: 85
  Number of functions evaluations: 108


INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 1279.671642
  Number of iterations: 85
  Number of functions evaluations: 108


In [ ]:
active(20, 0, 0, 1)